In [1]:
from concurrent.futures import ThreadPoolExecutor
import requests
import os
import glob
from Parsers.parsers import TURBD

In [ ]:
def download_file(url):
    response = requests.get(url)
    if "content-disposition" in response.headers:
        content_disposition = response.headers["content-disposition"]
        filename = content_disposition.split("filename=")[1]
    else:
        filename = url.split("/")[-1]
    with open(filename, mode="wb") as file:
        file.write(response.content)
    print(f"Downloaded file {filename}")

In [2]:
url = "https://rawdata-west.oceanobservatories.org/files/AS03CPSM/R00001/cg_data/dcl16/turbd/20230109.turbd.log"
response = requests.get(url)
response.url

'https://rawdata-west.oceanobservatories.org/files/AS03CPSM/R00001/cg_data/dcl16/turbd/20230109.turbd.log'

In [3]:
response.status_code

200

In [6]:
os.makedirs("data/ast3_turbidity_data", exist_ok=True)
with open("data/ast3_turbidity_data/{filename}".format(filename=response.url.split('/')[-1]), mode="wb") as file:
    file.write(response.content)

In [2]:
files = glob.glob("data/ast3_turbidity_data/*")
files

['data/ast3_turbidity_data/20230109.turbd.log']

In [8]:
turbd = TURBD()
turbd.load_turbd(files)
data = turbd.DATA

Parsing 20230109.turbd.log


In [9]:
data

,LGR_TIMESTAMP,TIMESTAMP,TURBIDITY
0,2023-01-09 00:00:05.674,2023-01-09 00:00:03.500,6.1616
1,2023-01-09 00:00:06.175,2023-01-09 00:00:04.000,6.1662
2,2023-01-09 00:00:06.674,2023-01-09 00:00:04.500,6.1648
3,2023-01-09 00:00:07.174,2023-01-09 00:00:05.000,6.1640
4,2023-01-09 00:00:07.675,2023-01-09 00:00:05.500,6.1633
...,...,...,...
5664,2023-01-09 23:31:02.230,2023-01-09 23:31:00.000,6.2734
5665,2023-01-09 23:31:02.733,2023-01-09 23:31:00.500,6.2758
5666,2023-01-09 23:31:03.231,2023-01-09 23:31:01.000,6.2758
5667,2023-01-09 23:31:03.730,2023-01-09 23:31:01.500,6.2749
